In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

def dataframe_to_table(df):
    table = pa.Table.from_pandas(df)
    return table

def table_to_dataframe(table):
    df = table.to_pandas()
    return df

def table_to_parquet(table, parquet_file):
    pq.write_table(table, parquet_file)

def parquet_to_table(parquet_file):
    table = pq.read_table(parquet_file)
    return table


def display_table_schema(table):
    schema = table.schema
    print("Table Schema:")
    for field in schema:
        print(f"{field.name}: {field.type}")


def get_column(table, col_name):
    try:
        selected_table = table.select([col_name])
        column = selected_table.to_pandas()[col_name]
        column.name = col_name

        return column
    except KeyError:
        print(f"La colonne {col_name} n'existe pas dans la table.")
        return None
    
def compute_statistics(table, col_name):
    col = table[col_name]
    count_result = pc.count(col)
    count_distinct_result = pc.count_distinct(col)
    sum_result = pc.sum(col)
    min_result = pc.min(col)
    max_result = pc.max(col)

    # Obtenir les valeurs des résultats
    count_value = count_result.as_py()
    count_distinct_value = count_distinct_result.as_py()
    sum_value = sum_result.as_py()
    min_value = min_result.as_py()
    max_value = max_result.as_py()

    # Afficher les résultats
    print(f"Count: {count_value}")
    print(f"Count Distinct: {count_distinct_value}")
    print(f"Sum: {sum_value}")
    print(f"Min: {min_value}")
    print(f"Max: {max_value}")

    res = {
        'Count'  : count_value,
        'Count Distinct' : count_distinct_result,
        'Sum' :  sum_value,
        'Min' : min_value,
        'Max ' : max_value

    }

    return pd.DataFrame([res])

## Question 1

In [2]:
from fonction import *

df_academie = pd.read_csv("academies_virgule.csv")
df_academie

df_villes = pd.read_csv("villes_virgule.csv")
table_villes = dataframe_to_table(df_villes)
parquet_ville_file = 'ville.parquet'
table_to_parquet(table_villes, parquet_ville_file) 

In [3]:
table_academie = dataframe_to_table(df_academie)
table_academie

pyarrow.Table
academie: string
wikipedia: string
vacances: string
dep: string
departement: string
region: string
----
academie: [["Académie de Lyon","Académie d'Amiens","Académie de Clermont-Ferrand","Académie d'Aix-Marseille","Académie d'Aix-Marseille",...,"Académie de Versailles","Académie de la Guadeloupe","Académie de la Martinique","Académie de Guyane","Académie de la Réunion"]]
wikipedia: [["fr:Académie de Lyon (éducation)","fr:Académie d'Amiens (éducation)","fr:Académie de Clermont-Ferrand","fr:Académie d'Aix-Marseille","fr:Académie d'Aix-Marseille",...,"fr:Académie de Versailles (éducation)","fr:Académie de la Guadeloupe","fr:Académie de la Martinique","fr:Académie de la Guyane","fr:Académie de La Réunion"]]
vacances: [["Zone A","Zone B","Zone A","Zone B","Zone B",...,"Zone C",null,null,null,null]]
dep: [["01","02","03","04","05",...,"95","971","972","973","974"]]
departement: [["Ain","Aisne","Allier","Alpes-de-Haute-Provence","Hautes-Alpes",...,"Val-d'Oise","Guadeloupe","Marti

In [4]:
parquet_file = 'academie.parquet'
table_to_parquet(table_academie, parquet_file)

In [5]:
parquet_to_table('academie.parquet')

pyarrow.Table
academie: string
wikipedia: string
vacances: string
dep: string
departement: string
region: string
----
academie: [["Académie de Lyon","Académie d'Amiens","Académie de Clermont-Ferrand","Académie d'Aix-Marseille","Académie d'Aix-Marseille",...,"Académie de Versailles","Académie de la Guadeloupe","Académie de la Martinique","Académie de Guyane","Académie de la Réunion"]]
wikipedia: [["fr:Académie de Lyon (éducation)","fr:Académie d'Amiens (éducation)","fr:Académie de Clermont-Ferrand","fr:Académie d'Aix-Marseille","fr:Académie d'Aix-Marseille",...,"fr:Académie de Versailles (éducation)","fr:Académie de la Guadeloupe","fr:Académie de la Martinique","fr:Académie de la Guyane","fr:Académie de La Réunion"]]
vacances: [["Zone A","Zone B","Zone A","Zone B","Zone B",...,"Zone C",null,null,null,null]]
dep: [["01","02","03","04","05",...,"95","971","972","973","974"]]
departement: [["Ain","Aisne","Allier","Alpes-de-Haute-Provence","Hautes-Alpes",...,"Val-d'Oise","Guadeloupe","Marti

## Question 2

In [6]:
schema_table = display_table_schema(table_academie)

Table Schema:
academie: string
wikipedia: string
vacances: string
dep: string
departement: string
region: string


In [7]:
get_column(table_academie, "dep")

0       01
1       02
2       03
3       04
4       05
      ... 
96      95
97     971
98     972
99     973
100    974
Name: dep, Length: 101, dtype: object

## Question 4

In [8]:
df_villes

,dep,nom,cp,nb_hab_2010,nb_hab_1999,nb_hab_2012,dens,surf,long,lat,alt_min,alt_max
0,1,Ozan,1190,618,469,500,93,6.60,4.91667,46.38330,170.0,205.0
1,1,Cormoranche-sur-Saône,1290,1058,903,1000,107,9.85,4.83333,46.23330,168.0,211.0
2,1,Plagne,1130,129,83,100,20,6.20,5.73333,46.18330,560.0,922.0
3,1,Tossiat,1250,1406,1111,1400,138,10.17,5.31667,46.13330,244.0,501.0
4,1,Pouillat,1250,88,58,100,14,6.23,5.43333,46.33330,333.0,770.0
...,...,...,...,...,...,...,...,...,...,...,...,...
36695,976,Sada,97640,10195,10195,10195,933,10.92,45.10470,-12.84860,NaN,NaN
36696,976,Tsingoni,97680,10454,10454,10454,300,34.76,45.10700,-12.78970,NaN,NaN
36697,971,Saint-Barthélemy,97133,8938,8938,8938,372,24.00,-62.83330,17.91670,NaN,NaN
36698,971,Saint-Martin,97150,36979,36979,36979,695,53.20,18.09130,-63.08290,NaN,NaN


In [9]:
compute_statistics(table_villes, 'dens')

Count: 36700
Count Distinct: 1663
Sum: 5688355
Min: 0
Max: 26660


In [10]:
compute_statistics(table_villes, 'surf')

Count: 36700
Count Distinct: 5190
Sum: 633345.6599999999
Min: 0.04
Max: 18360.0


## Question 6

### Calcul nombre moyen d'habitant en 2012

In [11]:
df_hab_2012 = df_villes['nb_hab_2012']
df_hab_2012.mean()

1751.080272479564

### Calcul du nombre moyen d'habitants par département

In [12]:
df = df_villes.groupby('dep')['nb_hab_2012'].mean()
df.head()

dep
1      1388.305489
10      696.304850
11      796.575342
12      905.263158
13    16515.966387
Name: nb_hab_2012, dtype: float64

### afficher le résultat pour le département 74

In [13]:
df_villes[df_villes['dep'] == '74']

,dep,nom,cp,nb_hab_2010,nb_hab_1999,nb_hab_2012,dens,surf,long,lat,alt_min,alt_max
30143,74,Andilly,74350,778,594,800,128,6.07,6.06667,46.0500,577.0,857.0
30144,74,Minzier,74270,862,498,800,98,8.79,5.98333,46.0500,456.0,750.0
30145,74,Saint-Paul-en-Chablais,74500,2172,1700,2100,150,14.45,6.63333,46.3833,698.0,1268.0
30146,74,Hauteville-sur-Fier,74150,825,685,800,168,4.90,5.98333,45.9000,320.0,524.0
30147,74,Vanzy,74270,303,242,300,54,5.57,5.88333,46.0500,287.0,537.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30432,74,Évires,74570,1294,1062,1300,66,19.49,6.23333,46.0333,593.0,955.0
30433,74,Chens-sur-Léman,74140,1893,1274,1700,174,10.87,6.26667,46.3333,372.0,439.0
30434,74,Quintal,74600,1198,882,1100,131,9.13,6.08333,45.8333,585.0,1533.0
30435,74,Ambilly,74100,5890,5811,5900,4712,1.25,6.22417,46.1950,410.0,434.0


In [14]:
mask = pc.equal(table_villes['dep'], pa.scalar('74'))
table_villes.filter(mask)

pyarrow.Table
dep: string
nom: string
cp: string
nb_hab_2010: int64
nb_hab_1999: int64
nb_hab_2012: int64
dens: int64
surf: double
long: double
lat: double
alt_min: double
alt_max: double
----
dep: [["74","74","74","74","74",...,"74","74","74","74","74"]]
nom: [["Andilly","Minzier","Saint-Paul-en-Chablais","Hauteville-sur-Fier","Vanzy",...,"Évires","Chens-sur-Léman","Quintal","Ambilly","Talloires"]]
cp: [["74350","74270","74500","74150","74270",...,"74570","74140","74600","74100","74290"]]
nb_hab_2010: [[778,862,2172,825,303,...,1294,1893,1198,5890,1690]]
nb_hab_1999: [[594,498,1700,685,242,...,1062,1274,882,5811,1447]]
nb_hab_2012: [[800,800,2100,800,300,...,1300,1700,1100,5900,1600]]
dens: [[128,98,150,168,54,...,66,174,131,4712,81]]
surf: [[6.07,8.79,14.45,4.9,5.57,...,19.49,10.87,9.13,1.25,20.69]]
long: [[6.06667,5.98333,6.63333,5.98333,5.88333,...,6.23333,6.26667,6.08333,6.22417,6.21667]]
lat: [[46.05,46.05,46.3833,45.9,46.05,...,46.0333,46.3333,45.8333,46.195,45.85]]
...